In [1]:
from ipywidgets import FloatSlider, Image, fixed, interactive
from IPython.display import display
from prompt import *
from local_kg import *
from config import init_configures
from utils import *

alpha_, beta_, freq_pen, hub_bias, prompt_model, story_model = init_configures()

inte1 = interactive(set_alpha, alpha=FloatSlider(min=0.0, max=1.0, step=0.05, value=alpha_[0]), alpha_=fixed(alpha_))
inte2 = interactive(set_beta, beta=FloatSlider(min=0.0, max=50.0, step=0.1, value=beta_[0]), beta_=fixed(beta_))
inte3 = interactive(set_freq_pen, freq_penalty=FloatSlider(min=0.0, max=2.0, step=0.05, value=freq_pen[0]), freq_pen=fixed(freq_pen))
inte4 = interactive(set_hub_word_bias, hub_word_bias=FloatSlider(min=1.0, max=30.0, step=1.0, value=hub_bias[0]), hub_bias=fixed(hub_bias))

prompt_button, story_button = set_models(prompt_model, story_model)
display(HBox([VBox([inte1, inte2, inte3, inte4]), prompt_button, story_button]))

input_box, inp, button = set_input_bar()
display(input_box)

widgetsHandler, logger = set_logging()
widgetsHandler.show_logs()

outputBoxes, box1, box2 = set_outputBox()
display(outputBoxes)

imageBoxes, imageOutPut = set_imageBox()
display(imageBoxes)

def onchange(b):
    widgetsHandler.clear_logs()
    box1.clear_output()
    box2.clear_output()
    imageOutPut.clear_output()

    show_parameters(logger, alpha_[0], beta_[0], freq_pen[0], prompt_model[0], story_model[0], hub_bias[0])

    kw_list, promt_sentence = promptGeneration(logger, inp.value, prompt_model[0])

    path, hubs, stem_to_words, nei_to_hub = build_kg(kw_list=kw_list)
    final_score = calculate_score(logger, path, hubs, stem_to_words, nei_to_hub, alpha_[0])
    logger.info(f"promt_sentence: \x1b[32;1m{promt_sentence}\x1b[0m")
    story = gpt3_generation(logger, kw_list, promt_sentence, final_score, stem_to_words, beta_[0], freq_pen[0], story_model[0], hub_bias[0])
    with box1:
        print(story)
    gpt3_story = original_gpt3(logger, kw_list, story_model[0])
    with box2:
        print(gpt3_story)
    
    logger.info("[drawing keywords graph]")
    with imageOutPut:
        with open("local_kgs/kg.png", "rb") as file:
            image = file.read()
        display(Image(value=image,
                    format='png',
                    layout={'width': '20cm',
                            'object_position': 'top'}))
    logger.info("\n")

button.on_click(onchange)

Output(layout=Layout(border='1px solid black', height='300px', margin='10px 50px 10px 50px', overflow='hidden …